In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import re
import time

/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:471: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:472: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:473: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:474: FutureWarning: Passing (type, 1) or 

Import

In [ ]:
lines = open('movie_lines.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')


In [ ]:
conversations = open('movie_conversations.txt', encoding = 'utf-8', errors = 'ignore').read().split('\n')
 

In [ ]:
lines

['L1045 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ They do not!',
 'L1044 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ They do to!',
 'L985 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I hope so.',
 'L984 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ She okay?',
 "L925 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Let's go.",
 'L924 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ Wow',
 "L872 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Okay -- you're gonna need to learn how to lie.",
 'L871 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ No',
 'L870 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ Like my fear of wearing pastels?',
 'L868 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ The "real you".',
 'L867 +++$+++ u0 +++$+++ m0 +++$+++ BIANCA +++$+++ What good stuff?',
 "L866 +++$+++ u2 +++$+++ m0 +++$+++ CAMERON +++$+++ I figured yo

In [ ]:
conversations

["u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L194', 'L195', 'L196', 'L197']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L198', 'L199']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L200', 'L201', 'L202', 'L203']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L204', 'L205', 'L206']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L207', 'L208']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L271', 'L272', 'L273', 'L274', 'L275']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L276', 'L277']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L280', 'L281']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L363', 'L364']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L365', 'L366']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L367', 'L368']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L401', 'L402', 'L403']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L404', 'L405', 'L406', 'L407']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L575', 'L576']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L577', 'L578']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L662', 'L663']",
 "u0 +++$+++ u2 +++$+++ m0 +++$+++ ['L693', 'L69

create dictionary that maps each line and its id

dictionary intialized

In [ ]:
id2line= {}

In [ ]:
for line in lines:
    _line = line.split(' +++$+++ ')
    if len(_line) == 5:
        id2line[_line[0]] = _line[4]

In [ ]:
id2line

{'L1045': 'They do not!',
 'L1044': 'They do to!',
 'L985': 'I hope so.',
 'L984': 'She okay?',
 'L925': "Let's go.",
 'L924': 'Wow',
 'L872': "Okay -- you're gonna need to learn how to lie.",
 'L871': 'No',
 'L870': 'I\'m kidding.  You know how sometimes you just become this "persona"?  And you don\'t know how to quit?',
 'L869': 'Like my fear of wearing pastels?',
 'L868': 'The "real you".',
 'L867': 'What good stuff?',
 'L866': "I figured you'd get to the good stuff eventually.",
 'L865': 'Thank God!  If I had to hear one more story about your coiffure...',
 'L864': "Me.  This endless ...blonde babble. I'm like, boring myself.",
 'L863': 'What crap?',
 'L862': 'do you listen to this crap?',
 'L861': 'No...',
 'L860': 'Then Guillermo says, "If you go any lighter, you\'re gonna look like an extra on 90210."',
 'L699': 'You always been this selfish?',
 'L698': 'But',
 'L697': "Then that's all you had to say.",
 'L696': 'Well, no...',
 'L695': "You never wanted to go out with 'me, did y

create a list of all the conversations

In [ ]:
conversations_ids = []
for conversation in conversations[:-1]:
    _conversation = conversation.split(' +++$+++ ')[-1][1:-1].replace("'", "").replace(" ", "")
    conversations_ids.append(_conversation.split(','))
  


In [ ]:
conversations_ids

[['L194', 'L195', 'L196', 'L197'],
 ['L198', 'L199'],
 ['L200', 'L201', 'L202', 'L203'],
 ['L204', 'L205', 'L206'],
 ['L207', 'L208'],
 ['L271', 'L272', 'L273', 'L274', 'L275'],
 ['L276', 'L277'],
 ['L280', 'L281'],
 ['L363', 'L364'],
 ['L365', 'L366'],
 ['L367', 'L368'],
 ['L401', 'L402', 'L403'],
 ['L404', 'L405', 'L406', 'L407'],
 ['L575', 'L576'],
 ['L577', 'L578'],
 ['L662', 'L663'],
 ['L693', 'L694', 'L695'],
 ['L696', 'L697', 'L698', 'L699'],
 ['L860', 'L861'],
 ['L862', 'L863', 'L864', 'L865'],
 ['L866', 'L867', 'L868', 'L869'],
 ['L870', 'L871', 'L872'],
 ['L924', 'L925'],
 ['L984', 'L985'],
 ['L1044', 'L1045'],
 ['L49', 'L50', 'L51'],
 ['L571', 'L572', 'L573'],
 ['L579', 'L580'],
 ['L595', 'L596', 'L597'],
 ['L598', 'L599', 'L600'],
 ['L659', 'L660'],
 ['L952', 'L953'],
 ['L394', 'L395'],
 ['L396', 'L397'],
 ['L589', 'L590', 'L591'],
 ['L592', 'L593'],
 ['L756', 'L757', 'L758'],
 ['L759', 'L760'],
 ['L164', 'L165'],
 ['L319', 'L320'],
 ['L441', 'L442', 'L443', 'L444', 'L445']

Get qn and answers or input and output

In [ ]:
questions = []
answers = []
for conversation in conversations_ids:
    for i in range(len(conversation) - 1):
        questions.append(id2line[conversation[i]])
        answers.append(id2line[conversation[i+1]])

In [ ]:
answers

["Well, I thought we'd start with pronunciation, if that's okay with you.",
 'Not the hacking and gagging and spitting part.  Please.',
 "Okay... then how 'bout we try out some French cuisine.  Saturday?  Night?",
 'Forget it.',
 'Cameron.',
 "The thing is, Cameron -- I'm at the mercy of a particularly hideous breed of loser.  My sister.  I can't date until she does.",
 'Seems like she could get a date easy enough...',
 'Unsolved mystery.  She used to be really popular when she started high school, then it was just like she got sick of it or something.',
 "That's a shame.",
 'Let me see what I can do.',
 "Right.  See?  You're ready for the quiz.",
 "I don't want to know how to say that though.  I want to know useful things. Like where the good stores are.  How much does champagne cost?  Stuff like Chat.  I have never in my life had to point out my head to someone.",
 "That's because it's such a nice one.",
 'Forget French.',
 "Well, there's someone I think might be --",
 'Where?',
 "I 

Data cleaning

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "what is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    return text

cleaning the questions

In [ ]:
clean_questions = []
for question in questions:
    clean_questions.append(clean_text(question))

clean answers

In [ ]:
clean_answers = []
for answer in answers:
    clean_answers.append(clean_text(answer))

In [ ]:
clean_answers

['well i thought we would start with pronunciation if that is okay with you',
 'not the hacking and gagging and spitting part  please',
 "okay then how 'bout we try out some french cuisine  saturday  night",
 'forget it',
 'cameron',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i ca not date until she does',
 'seems like she could get a date easy enough',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something',
 'that is a shame',
 'let me see what i can do',
 'right  see  you are ready for the quiz',
 'i do not want to know how to say that though  i want to know useful things like where the good stores are  how much does champagne cost  stuff like chat  i have never in my life had to point out my head to someone',
 "that is because it's such a nice one",
 'forget french',
 "well there's someone i think might be ",
 'where',
 'i counted on you to help my cause

create a dict that maps each word to its number of occurences

In [ ]:
word2count = {}
for question in clean_questions:
    for word in question.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1
for answer in clean_answers:
    for word in answer.split():
        if word not in word2count:
            word2count[word] = 1
        else:
            word2count[word] += 1


In [ ]:
word2count

{'can': 15996,
 'we': 40514,
 'make': 6751,
 'this': 33873,
 'quick': 359,
 'roxanne': 1,
 'korrine': 1,
 'and': 65608,
 'andrew': 56,
 'barrett': 20,
 'are': 56142,
 'having': 1217,
 'an': 9482,
 'incredibly': 60,
 'horrendous': 4,
 'public': 369,
 'break': 918,
 'up': 16567,
 'on': 27568,
 'the': 140648,
 'quad': 2,
 'again': 3322,
 'well': 14173,
 'i': 204570,
 'thought': 4559,
 'would': 23032,
 'start': 1658,
 'with': 24986,
 'pronunciation': 2,
 'if': 18959,
 'that': 67605,
 'is': 84002,
 'okay': 6184,
 'you': 214123,
 'not': 109722,
 'hacking': 18,
 'gagging': 9,
 'spitting': 16,
 'part': 1437,
 'please': 3429,
 'asking': 748,
 'me': 46144,
 'out': 18894,
 'so': 19103,
 'cute': 281,
 'what': 55624,
 'your': 29938,
 'name': 3153,
 'no': 28710,
 "it's": 25845,
 'my': 29707,
 'fault': 514,
 'did': 21511,
 'have': 48703,
 'a': 102020,
 'proper': 141,
 'introduction': 19,
 'cameron': 35,
 'thing': 5822,
 'am': 37918,
 'at': 15298,
 'mercy': 75,
 'of': 56307,
 'particularly': 111,
 'hi

Creating two dictionaries that map the questions words and the answers words to a unique integer

In [ ]:
word2count.items()

dict_items([('can', 15996), ('we', 40514), ('make', 6751), ('this', 33873), ('quick', 359), ('roxanne', 1), ('korrine', 1), ('and', 65608), ('andrew', 56), ('barrett', 20), ('are', 56142), ('having', 1217), ('an', 9482), ('incredibly', 60), ('horrendous', 4), ('public', 369), ('break', 918), ('up', 16567), ('on', 27568), ('the', 140648), ('quad', 2), ('again', 3322), ('well', 14173), ('i', 204570), ('thought', 4559), ('would', 23032), ('start', 1658), ('with', 24986), ('pronunciation', 2), ('if', 18959), ('that', 67605), ('is', 84002), ('okay', 6184), ('you', 214123), ('not', 109722), ('hacking', 18), ('gagging', 9), ('spitting', 16), ('part', 1437), ('please', 3429), ('asking', 748), ('me', 46144), ('out', 18894), ('so', 19103), ('cute', 281), ('what', 55624), ('your', 29938), ('name', 3153), ('no', 28710), ("it's", 25845), ('my', 29707), ('fault', 514), ('did', 21511), ('have', 48703), ('a', 102020), ('proper', 141), ('introduction', 19), ('cameron', 35), ('thing', 5822), ('am', 3791

In [ ]:
len(word2count)

69051

In [ ]:
threshold_questions = 20
questionswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold_questions:
        questionswords2int[word] = word_number
        word_number += 1
threshold_answers = 20
answerswords2int = {}
word_number = 0
for word, count in word2count.items():
    if count >= threshold_answers:
        answerswords2int[word] = word_number
        word_number += 1  

In [ ]:
questionswords2int

{'can': 0,
 'we': 1,
 'make': 2,
 'this': 3,
 'quick': 4,
 'and': 5,
 'andrew': 6,
 'barrett': 7,
 'are': 8,
 'having': 9,
 'an': 10,
 'incredibly': 11,
 'public': 12,
 'break': 13,
 'up': 14,
 'on': 15,
 'the': 16,
 'again': 17,
 'well': 18,
 'i': 19,
 'thought': 20,
 'would': 21,
 'start': 22,
 'with': 23,
 'if': 24,
 'that': 25,
 'is': 26,
 'okay': 27,
 'you': 28,
 'not': 29,
 'part': 30,
 'please': 31,
 'asking': 32,
 'me': 33,
 'out': 34,
 'so': 35,
 'cute': 36,
 'what': 37,
 'your': 38,
 'name': 39,
 'no': 40,
 "it's": 41,
 'my': 42,
 'fault': 43,
 'did': 44,
 'have': 45,
 'a': 46,
 'proper': 47,
 'cameron': 48,
 'thing': 49,
 'am': 50,
 'at': 51,
 'mercy': 52,
 'of': 53,
 'particularly': 54,
 'hideous': 55,
 'breed': 56,
 'loser': 57,
 'sister': 58,
 'ca': 59,
 'date': 60,
 'until': 61,
 'she': 62,
 'does': 63,
 'why': 64,
 'mystery': 65,
 'used': 66,
 'to': 67,
 'be': 68,
 'really': 69,
 'popular': 70,
 'when': 71,
 'started': 72,
 'high': 73,
 'school': 74,
 'then': 75,
 'it':

In [ ]:
len(questionswords2int)

8611

In [ ]:
answerswords2int

{'can': 0,
 'we': 1,
 'make': 2,
 'this': 3,
 'quick': 4,
 'and': 5,
 'andrew': 6,
 'barrett': 7,
 'are': 8,
 'having': 9,
 'an': 10,
 'incredibly': 11,
 'public': 12,
 'break': 13,
 'up': 14,
 'on': 15,
 'the': 16,
 'again': 17,
 'well': 18,
 'i': 19,
 'thought': 20,
 'would': 21,
 'start': 22,
 'with': 23,
 'if': 24,
 'that': 25,
 'is': 26,
 'okay': 27,
 'you': 28,
 'not': 29,
 'part': 30,
 'please': 31,
 'asking': 32,
 'me': 33,
 'out': 34,
 'so': 35,
 'cute': 36,
 'what': 37,
 'your': 38,
 'name': 39,
 'no': 40,
 "it's": 41,
 'my': 42,
 'fault': 43,
 'did': 44,
 'have': 45,
 'a': 46,
 'proper': 47,
 'cameron': 48,
 'thing': 49,
 'am': 50,
 'at': 51,
 'mercy': 52,
 'of': 53,
 'particularly': 54,
 'hideous': 55,
 'breed': 56,
 'loser': 57,
 'sister': 58,
 'ca': 59,
 'date': 60,
 'until': 61,
 'she': 62,
 'does': 63,
 'why': 64,
 'mystery': 65,
 'used': 66,
 'to': 67,
 'be': 68,
 'really': 69,
 'popular': 70,
 'when': 71,
 'started': 72,
 'high': 73,
 'school': 74,
 'then': 75,
 'it':

In [ ]:
len(answerswords2int)

8611

 Adding the last tokens to these two dictionaries

In [ ]:
tokens = ['<PAD>', '<EOS>', '<OUT>', '<SOS>']
for token in tokens:
  questionswords2int[token]=len(questionswords2int)+1
for token in tokens:
  answerswords2int[token]=len(answerswords2int)+1  

In [ ]:
answerswords2int[token]

8615

In [ ]:
questionswords2int[token]

8616

 Creating the inverse dictionary of the answerswords2int dictionary

In [ ]:
answersints2word = {w_i: w for w, w_i in answerswords2int.items()}

In [ ]:
answersints2word

{0: 'can',
 1: 'we',
 2: 'make',
 3: 'this',
 4: 'quick',
 5: 'and',
 6: 'andrew',
 7: 'barrett',
 8: 'are',
 9: 'having',
 10: 'an',
 11: 'incredibly',
 12: 'public',
 13: 'break',
 14: 'up',
 15: 'on',
 16: 'the',
 17: 'again',
 18: 'well',
 19: 'i',
 20: 'thought',
 21: 'would',
 22: 'start',
 23: 'with',
 24: 'if',
 25: 'that',
 26: 'is',
 27: 'okay',
 28: 'you',
 29: 'not',
 30: 'part',
 31: 'please',
 32: 'asking',
 33: 'me',
 34: 'out',
 35: 'so',
 36: 'cute',
 37: 'what',
 38: 'your',
 39: 'name',
 40: 'no',
 41: "it's",
 42: 'my',
 43: 'fault',
 44: 'did',
 45: 'have',
 46: 'a',
 47: 'proper',
 48: 'cameron',
 49: 'thing',
 50: 'am',
 51: 'at',
 52: 'mercy',
 53: 'of',
 54: 'particularly',
 55: 'hideous',
 56: 'breed',
 57: 'loser',
 58: 'sister',
 59: 'ca',
 60: 'date',
 61: 'until',
 62: 'she',
 63: 'does',
 64: 'why',
 65: 'mystery',
 66: 'used',
 67: 'to',
 68: 'be',
 69: 'really',
 70: 'popular',
 71: 'when',
 72: 'started',
 73: 'high',
 74: 'school',
 75: 'then',
 76: '

Adding the End Of String(EOS) token to the end of every answer

In [ ]:
for i in range(len(clean_answers)):
  clean_answers[i]+= ' <EOS>'

In [ ]:
clean_answers

['well i thought we would start with pronunciation if that is okay with you <EOS>',
 'not the hacking and gagging and spitting part  please <EOS>',
 "okay then how 'bout we try out some french cuisine  saturday  night <EOS>",
 'forget it <EOS>',
 'cameron <EOS>',
 'the thing is cameron  i am at the mercy of a particularly hideous breed of loser  my sister  i ca not date until she does <EOS>',
 'seems like she could get a date easy enough <EOS>',
 'unsolved mystery  she used to be really popular when she started high school then it was just like she got sick of it or something <EOS>',
 'that is a shame <EOS>',
 'let me see what i can do <EOS>',
 'right  see  you are ready for the quiz <EOS>',
 'i do not want to know how to say that though  i want to know useful things like where the good stores are  how much does champagne cost  stuff like chat  i have never in my life had to point out my head to someone <EOS>',
 "that is because it's such a nice one <EOS>",
 'forget french <EOS>',
 "we

1. Translating all the questions and the answers into integers
2. and Replacing all the words that were filtered out by  using OUT

In [ ]:
questions_to_int=[]
for question in clean_questions:
    ints = []
    for word in question.split():
        if word not in questionswords2int:
            ints.append(questionswords2int['<OUT>'])
        else:
            ints.append(questionswords2int[word])
    questions_to_int.append(ints)  

In [ ]:
answers_to_int = []
for answer in clean_answers:
    ints = []
    for word in answer.split():
        if word not in answerswords2int:
            ints.append(answerswords2int['<OUT>'])
        else:
            ints.append(answerswords2int[word])
    answers_to_int.append(ints)

Sorting questions and answers by the length of questions

In [ ]:
sorted_clean_questions = []
sorted_clean_answers = []
for length in range(1, 25 + 1):
    for i in enumerate(questions_to_int):
        if len(i[1]) == length:
            sorted_clean_questions.append(questions_to_int[i[0]])
            sorted_clean_answers.append(answers_to_int[i[0]])

In [ ]:
sorted_clean_questions

[[48],
 [64],
 [125],
 [148],
 [136],
 [40],
 [176],
 [40],
 [181],
 [182],
 [220],
 [37],
 [64],
 [136],
 [64],
 [112],
 [368],
 [133],
 [294],
 [210],
 [37],
 [221],
 [181],
 [27],
 [64],
 [181],
 [8614],
 [446],
 [249],
 [181],
 [192],
 [8614],
 [8614],
 [656],
 [99],
 [8614],
 [8614],
 [37],
 [40],
 [37],
 [8614],
 [148],
 [92],
 [37],
 [37],
 [8614],
 [624],
 [40],
 [40],
 [926],
 [813],
 [1102],
 [40],
 [230],
 [71],
 [210],
 [40],
 [143],
 [1243],
 [210],
 [1094],
 [1094],
 [1094],
 [1094],
 [210],
 [334],
 [150],
 [27],
 [94],
 [656],
 [37],
 [1120],
 [1238],
 [1480],
 [8614],
 [1484],
 [210],
 [1522],
 [37],
 [37],
 [210],
 [1570],
 [1570],
 [1570],
 [1570],
 [1570],
 [1570],
 [27],
 [656],
 [69],
 [656],
 [230],
 [99],
 [1599],
 [1570],
 [1570],
 [1570],
 [8614],
 [1570],
 [1570],
 [69],
 [1639],
 [661],
 [1757],
 [210],
 [39],
 [210],
 [1202],
 [210],
 [1322],
 [210],
 [18],
 [125],
 [37],
 [64],
 [1806],
 [1739],
 [210],
 [1814],
 [210],
 [210],
 [1789],
 [221],
 [1128],
 [

model building

creating placeholders for inputs and targets

why placehholder-
in tensorflow all variables are tensors, more advanced array and help for faster computations. 

[None, None]= 2 dimensional

In [ ]:
def model_inputs():
    inputs = tf.placeholder(tf.int32, [None, None], name = 'input')
    targets =tf.placeholder(tf.int32, [None, None], name = 'target')
    lr = tf.placeholder(tf.float32, name = 'learning_rate')
    keep_prob = tf.placeholder(tf.float32, name = 'keep_prob')
    return inputs, targets, lr, keep_prob

Preprocessing the targets


decoder accept certain format, inorder to use in RNN

target must be into batches.

feed neural nets with batches

each batch should start with SOS, start of sentence

create batch and sos

In [ ]:
def preprocess_targets(targets, word2int, batch_size):
    left_side = tf.fill([batch_size, 1], word2int['<SOS>'])
    right_side = tf.strided_slice(targets, [0,0], [batch_size, -1], [1,1])
    preprocessed_targets = tf.concat([left_side, right_side], 1)
    return preprocessed_targets

Creating the Encoder RNN

In [ ]:
def encoder_rnn(rnn_inputs, rnn_size, num_layers, keep_prob, sequence_length):
    lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
    lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
    encoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
    encoder_output, encoder_state = tf.nn.bidirectional_dynamic_rnn(cell_fw = encoder_cell,
                                                                    cell_bw = encoder_cell,
                                                                    sequence_length = sequence_length,
                                                                    inputs = rnn_inputs,
                                                                    dtype = tf.float32)
    return encoder_state

Decoding the training set

In [ ]:
def decode_training_set(encoder_state, decoder_cell, decoder_embedded_input, sequence_length, decoding_scope, output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
    training_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_train(encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              name = "attn_dec_train")
    decoder_output, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                              training_decoder_function,
                                                                                                              decoder_embedded_input,
                                                                                                              sequence_length,
                                                                                                              scope = decoding_scope)
    decoder_output_dropout = tf.nn.dropout(decoder_output, keep_prob)
    return output_function(decoder_output_dropout)

Decoding the test/validation set

In [ ]:
def decode_test_set(encoder_state, decoder_cell, decoder_embeddings_matrix, sos_id, eos_id, maximum_length, num_words, decoding_scope, output_function, keep_prob, batch_size):
    attention_states = tf.zeros([batch_size, 1, decoder_cell.output_size])
    attention_keys, attention_values, attention_score_function, attention_construct_function = tf.contrib.seq2seq.prepare_attention(attention_states, attention_option = "bahdanau", num_units = decoder_cell.output_size)
    test_decoder_function = tf.contrib.seq2seq.attention_decoder_fn_inference(output_function,
                                                                              encoder_state[0],
                                                                              attention_keys,
                                                                              attention_values,
                                                                              attention_score_function,
                                                                              attention_construct_function,
                                                                              decoder_embeddings_matrix,
                                                                              sos_id,
                                                                              eos_id,
                                                                              maximum_length,
                                                                              num_words,
                                                                              name = "attn_dec_inf")
    test_predictions, decoder_final_state, decoder_final_context_state = tf.contrib.seq2seq.dynamic_rnn_decoder(decoder_cell,
                                                                                                                test_decoder_function,
                                                                                                                scope = decoding_scope)
    return test_predictions

Creating the Decoder RNN

In [ ]:
def decoder_rnn(decoder_embedded_input, decoder_embeddings_matrix, encoder_state, num_words, sequence_length, rnn_size, num_layers, word2int, keep_prob, batch_size):
    with tf.variable_scope("decoding") as decoding_scope:
        lstm = tf.contrib.rnn.BasicLSTMCell(rnn_size)
        lstm_dropout = tf.contrib.rnn.DropoutWrapper(lstm, input_keep_prob = keep_prob)
        decoder_cell = tf.contrib.rnn.MultiRNNCell([lstm_dropout] * num_layers)
        weights = tf.truncated_normal_initializer(stddev = 0.1)
        biases = tf.zeros_initializer()
        output_function = lambda x: tf.contrib.layers.fully_connected(x,
                                                                      num_words,
                                                                      None,
                                                                      scope = decoding_scope,
                                                                      weights_initializer = weights,
                                                                      biases_initializer = biases)
        training_predictions = decode_training_set(encoder_state,
                                                   decoder_cell,
                                                   decoder_embedded_input,
                                                   sequence_length,
                                                   decoding_scope,
                                                   output_function,
                                                   keep_prob,
                                                   batch_size)
        decoding_scope.reuse_variables()
        test_predictions = decode_test_set(encoder_state,
                                           decoder_cell,
                                           decoder_embeddings_matrix,
                                           word2int['<SOS>'],
                                           word2int['<EOS>'],
                                           sequence_length - 1,
                                           num_words,
                                           decoding_scope,
                                           output_function,
                                           keep_prob,
                                           batch_size)
    return training_predictions, test_predictions

seq2seq model

In [ ]:
def seq2seq_model(inputs, targets, keep_prob, batch_size, sequence_length, answers_num_words, questions_num_words, encoder_embedding_size, decoder_embedding_size, rnn_size, num_layers, questionswords2int):
    encoder_embedded_input = tf.contrib.layers.embed_sequence(inputs,
                                                              answers_num_words + 1,
                                                              encoder_embedding_size,
                                                              initializer = tf.random_uniform_initializer(0, 1))
    encoder_state = encoder_rnn(encoder_embedded_input, rnn_size, num_layers, keep_prob, sequence_length)
    preprocessed_targets = preprocess_targets(targets, questionswords2int, batch_size)
    decoder_embeddings_matrix = tf.Variable(tf.random_uniform([questions_num_words + 1, decoder_embedding_size], 0, 1))
    decoder_embedded_input = tf.nn.embedding_lookup(decoder_embeddings_matrix, preprocessed_targets)
    training_predictions, test_predictions = decoder_rnn(decoder_embedded_input,
                                                         decoder_embeddings_matrix,
                                                         encoder_state,
                                                         questions_num_words,
                                                         sequence_length,
                                                         rnn_size,
                                                         num_layers,
                                                         questionswords2int,
                                                         keep_prob,
                                                         batch_size)
    return training_predictions, test_predictions

In [ ]:
 import tensorflow as tf  

TRAINING THE SEQ2SEQ MODEL

In [ ]:
tf.compat.v1

<module 'tensorflow._api.v1.compat.v1' from '/usr/local/lib/python3.6/dist-packages/tensorflow/_api/v1/compat/v1/__init__.py'>

In [ ]:
epochs = 100
batch_size = 32
rnn_size = 1024
num_layers = 3
encoding_embedding_size = 1024
decoding_embedding_size = 1024
learning_rate = 0.001
learning_rate_decay = 0.9
min_learning_rate = 0.0001
keep_probability = 0.5
 
# Defining a session
tf.reset_default_graph()
session = tf.InteractiveSession()
 
# Loading the model inputs
inputs, targets, lr, keep_prob = model_inputs()
 
# Setting the sequence length
sequence_length = tf.placeholder_with_default(25, None, name = 'sequence_length')
 
# Getting the shape of the inputs tensor
input_shape = tf.shape(inputs)
 
# Getting the training and test predictions
training_predictions, test_predictions = seq2seq_model(tf.reverse(inputs, [-1]),
                                                       targets,
                                                       keep_prob,
                                                       batch_size,
                                                       sequence_length,
                                                       len(answerswords2int),
                                                       len(questionswords2int),
                                                       encoding_embedding_size,
                                                       decoding_embedding_size,
                                                       rnn_size,
                                                       num_layers,
                                                       questionswords2int)
 

In [ ]:
pip uninstall tensorflow

Uninstalling tensorflow-2.3.0:
  Would remove:
    /usr/local/bin/estimator_ckpt_converter
    /usr/local/bin/saved_model_cli
    /usr/local/bin/tensorboard
    /usr/local/bin/tf_upgrade_v2
    /usr/local/bin/tflite_convert
    /usr/local/bin/toco
    /usr/local/bin/toco_from_protos
    /usr/local/lib/python3.6/dist-packages/tensorflow-2.3.0.dist-info/*
    /usr/local/lib/python3.6/dist-packages/tensorflow/*
Proceed (y/n)? y
y
  Successfully uninstalled tensorflow-2.3.0


In [ ]:
pip install tensorflow==1.0.0

In [ ]:
# Setting up the Loss Error, the Optimizer and Gradient Clipping
with tf.name_scope("optimization"):
    loss_error = tf.contrib.seq2seq.sequence_loss(training_predictions,
                                                  targets,
                                                  tf.ones([input_shape[0], sequence_length]))
    optimizer = tf.train.AdamOptimizer(learning_rate)
    gradients = optimizer.compute_gradients(loss_error)
    clipped_gradients = [(tf.clip_by_value(grad_tensor, -5., 5.), grad_variable) for grad_tensor, grad_variable in gradients if grad_tensor is not None]
    optimizer_gradient_clipping = optimizer.apply_gradients(clipped_gradients)
 

In [ ]:
# Padding the sequences with the <PAD> token
def apply_padding(batch_of_sequences, word2int):
    max_sequence_length = max([len(sequence) for sequence in batch_of_sequences])
    return [sequence + [word2int['<PAD>']] * (max_sequence_length - len(sequence)) for sequence in batch_of_sequences]

In [ ]:
# Splitting the data into batches of questions and answers
def split_into_batches(questions, answers, batch_size):
    for batch_index in range(0, len(questions) // batch_size):
        start_index = batch_index * batch_size
        questions_in_batch = questions[start_index : start_index + batch_size]
        answers_in_batch = answers[start_index : start_index + batch_size]
        padded_questions_in_batch = np.array(apply_padding(questions_in_batch, questionswords2int))
        padded_answers_in_batch = np.array(apply_padding(answers_in_batch, answerswords2int))
        yield padded_questions_in_batch, padded_answers_in_batch
 
# Splitting the questions and answers into training and validation sets
training_validation_split = int(len(sorted_clean_questions) * 0.15)
training_questions = sorted_clean_questions[training_validation_split:]
training_answers = sorted_clean_answers[training_validation_split:]
validation_questions = sorted_clean_questions[:training_validation_split]
validation_answers = sorted_clean_answers[:training_validation_split]

In [ ]:
# Training
batch_index_check_training_loss = 100
batch_index_check_validation_loss = ((len(training_questions)) // batch_size // 2) - 1
total_training_loss_error = 0
list_validation_loss_error = []
early_stopping_check = 0
early_stopping_stop = 100
checkpoint = "./chatbot_weights.ckpt"
session.run(tf.global_variables_initializer())
for epoch in range(1, epochs + 1):
    for batch_index, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(training_questions, training_answers, batch_size)):
        starting_time = time.time()
        _, batch_training_loss_error = session.run([optimizer_gradient_clipping, loss_error], {inputs: padded_questions_in_batch,
                                                                                               targets: padded_answers_in_batch,
                                                                                               lr: learning_rate,
                                                                                               sequence_length: padded_answers_in_batch.shape[1],
                                                                                               keep_prob: keep_probability})
        total_training_loss_error += batch_training_loss_error
        ending_time = time.time()
        batch_time = ending_time - starting_time
        if batch_index % batch_index_check_training_loss == 0:
            print('Epoch: {:>3}/{}, Batch: {:>4}/{}, Training Loss Error: {:>6.3f}, Training Time on 100 Batches: {:d} seconds'.format(epoch,
                                                                                                                                       epochs,
                                                                                                                                       batch_index,
                                                                                                                                       len(training_questions) // batch_size,
                                                                                                                                       total_training_loss_error / batch_index_check_training_loss,
                                                                                                                                       int(batch_time * batch_index_check_training_loss)))
            total_training_loss_error = 0
        if batch_index % batch_index_check_validation_loss == 0 and batch_index > 0:
            total_validation_loss_error = 0
            starting_time = time.time()
            for batch_index_validation, (padded_questions_in_batch, padded_answers_in_batch) in enumerate(split_into_batches(validation_questions, validation_answers, batch_size)):
                batch_validation_loss_error = session.run(loss_error, {inputs: padded_questions_in_batch,
                                                                       targets: padded_answers_in_batch,
                                                                       lr: learning_rate,
                                                                       sequence_length: padded_answers_in_batch.shape[1],
                                                                       keep_prob: 1})
                total_validation_loss_error += batch_validation_loss_error
            ending_time = time.time()
            batch_time = ending_time - starting_time
            average_validation_loss_error = total_validation_loss_error / (len(validation_questions) / batch_size)
            print('Validation Loss Error: {:>6.3f}, Batch Validation Time: {:d} seconds'.format(average_validation_loss_error, int(batch_time)))
            learning_rate *= learning_rate_decay
            if learning_rate < min_learning_rate:
                learning_rate = min_learning_rate
            list_validation_loss_error.append(average_validation_loss_error)
            if average_validation_loss_error <= min(list_validation_loss_error):
                print('I speak better now!!')
                early_stopping_check = 0
                saver = tf.train.Saver()
                saver.save(session, checkpoint)
            else:
                print("Sorry I do not speak better, I need to practice more.")
                early_stopping_check += 1
                if early_stopping_check == early_stopping_stop:
                    break
    if early_stopping_check == early_stopping_stop:
        print("My apologies, I cannot speak better anymore. This is the best I can do.")
        break
print("Game Over")
 

Epoch:   1/100, Batch:    0/5401, Training Loss Error:  0.092, Training Time on 100 Batches: 3132 seconds
Epoch:   1/100, Batch:  100/5401, Training Loss Error:  2.048, Training Time on 100 Batches: 1687 seconds
Epoch:   1/100, Batch:  200/5401, Training Loss Error:  1.515, Training Time on 100 Batches: 3803 seconds


TESTING THE SEQ2SEQ MODEL

In [ ]:
# Loading the weights and Running the session
checkpoint = "./chatbot_weights.ckpt"
session = tf.InteractiveSession()
session.run(tf.global_variables_initializer())
saver = tf.train.Saver()
saver.restore(session, checkpoint)
 
# Converting the questions from strings to lists of encoding integers
def convert_string2int(question, word2int):
    question = clean_text(question)
    return [word2int.get(word, word2int['<OUT>']) for word in question.split()]
 
# Setting up the chat
while(True):
    question = input("You: ")
    if question == 'Goodbye':
        break
    question = convert_string2int(question, questionswords2int)
    question = question + [questionswords2int['<PAD>']] * (25 - len(question))
    fake_batch = np.zeros((batch_size, 25))
    fake_batch[0] = question
    predicted_answer = session.run(test_predictions, {inputs: fake_batch, keep_prob: 0.5})[0]
    answer = ''
    for i in np.argmax(predicted_answer, 1):
        if answersints2word[i] == 'i':
            token = ' I'
        elif answersints2word[i] == '<EOS>':
            token = '.'
        elif answersints2word[i] == '<OUT>':
            token = 'out'
        else:
            token = ' ' + answersints2word[i]
        answer += token
        if token == '.':
            break
    print('ChatBot: ' + answer)

